# OASIS MRI Images EDA
Exploratory Data Analysis for OASIS-2 Longitudinal MRI Dataset.

In [ ]:
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Intentar importar nibabel
try:
    import nibabel as nib
    NIBABEL_AVAILABLE = True
except ImportError:
    print("⚠️  nibabel no está instalado. Instalando...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nibabel"])
    import nibabel as nib
    NIBABEL_AVAILABLE = True
    print("✓ nibabel instalado correctamente")

# Configurar rutas
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / "data" / "raw" / "OASIS" / "images"

# Verificar si el directorio existe
if not DATA_DIR.exists():
    print(f"⚠️  El directorio {DATA_DIR} no existe.")
    print(f"📁 Directorios disponibles en data/raw/images:")
    
    # Buscar directorios alternativos
    images_dir = BASE_DIR / "data" / "raw" / "images"
    if images_dir.exists():
        subdirs = [d for d in images_dir.iterdir() if d.is_dir()]
        if subdirs:
            print(f"   Directorios encontrados: {[d.name for d in subdirs]}")
            # Usar el primer directorio disponible o preguntar
            DATA_DIR = subdirs[0] if subdirs else None
            print(f"   Usando: {DATA_DIR}")
        else:
            print(f"   El directorio {images_dir} está vacío")
            DATA_DIR = None
    else:
        print(f"   El directorio {images_dir} no existe")
        DATA_DIR = None
else:
    print(f"✓ Directorio encontrado: {DATA_DIR}")

# Listar sujetos si el directorio existe
if DATA_DIR and DATA_DIR.exists():
    try:
        subjects = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
        if subjects:
            print(f"✓ Encontrados {len(subjects)} sujetos")
            print(f"   Primeros 10: {subjects[:10]}")
        else:
            print("⚠️  No se encontraron subdirectorios de sujetos")
            subjects = []
    except Exception as e:
        print(f"❌ Error al listar directorio: {e}")
        subjects = []
else:
    print("❌ No se puede continuar sin un directorio válido de datos")
    subjects = []

In [ ]:
# Load one subject MRI
if not subjects:
    print("❌ No hay sujetos disponibles. Por favor, verifica que los datos estén en el directorio correcto.")
    print(f"   Directorio esperado: {BASE_DIR / 'data' / 'raw' / 'OASIS' / 'images'}")
    img = None
else:
    try:
        sample_subj = subjects[0]
        subj_path = Path(DATA_DIR) / sample_subj
        
        if not subj_path.exists():
            print(f"❌ El directorio del sujeto no existe: {subj_path}")
            img = None
        else:
            # Buscar archivos NIfTI
            files = [f for f in os.listdir(subj_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
            
            if not files:
                print(f"⚠️  No se encontraron archivos .nii o .nii.gz en {subj_path}")
                print(f"   Archivos encontrados: {os.listdir(subj_path)[:5]}")
                img = None
            else:
                sample_img_path = subj_path / files[0]
                print(f"✓ Cargando imagen: {sample_img_path}")
                
                try:
                    img = nib.load(str(sample_img_path)).get_fdata()
                    print(f"✓ Imagen cargada exitosamente")
                    print(f"   Shape: {img.shape}")
                    print(f"   Tipo: {img.dtype}")
                    print(f"   Rango: [{img.min():.2f}, {img.max():.2f}]")
                except Exception as e:
                    print(f"❌ Error al cargar la imagen: {e}")
                    img = None
    except Exception as e:
        print(f"❌ Error: {e}")
        img = None

In [ ]:
# Show axial, sagittal, coronal cuts
if img is not None:
    try:
        fig, axes = plt.subplots(1, 3, figsize=(15,5))
        
        # Asegurarse de que la imagen tiene 3 dimensiones
        if len(img.shape) == 3:
            axes[0].imshow(img[img.shape[0]//2,:,:], cmap='gray')
            axes[0].set_title('Sagittal')
            
            axes[1].imshow(img[:,img.shape[1]//2,:], cmap='gray')
            axes[1].set_title('Coronal')
            
            axes[2].imshow(img[:,:,img.shape[2]//2], cmap='gray')
            axes[2].set_title('Axial')
            
            plt.tight_layout()
            plt.show()
            print("✓ Visualización completada")
        else:
            print(f"⚠️  La imagen no tiene 3 dimensiones. Shape: {img.shape}")
    except Exception as e:
        print(f"❌ Error al visualizar: {e}")
else:
    print("⚠️  No hay imagen disponible para visualizar")

In [ ]:
# Intensity histogram
if img is not None:
    try:
        plt.figure(figsize=(10, 6))
        plt.hist(img.ravel(), bins=200, color='steelblue', edgecolor='black', alpha=0.7)
        plt.title('Intensity Histogram', fontsize=14, fontweight='bold')
        plt.xlabel('Intensity', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Estadísticas adicionales
        print(f"\n📊 Estadísticas de intensidad:")
        print(f"   Media: {img.mean():.2f}")
        print(f"   Mediana: {np.median(img):.2f}")
        print(f"   Desviación estándar: {img.std():.2f}")
        print(f"   Mínimo: {img.min():.2f}")
        print(f"   Máximo: {img.max():.2f}")
    except Exception as e:
        print(f"❌ Error al crear histograma: {e}")
else:
    print("⚠️  No hay imagen disponible para el histograma")